In [ ]:
import sqlite3 as sq

In [ ]:
conn = sq.connect('space_tourism.db')
cursor = conn.cursor()

In [ ]:

try:
    # Create tables in proper order to handle foreign key dependencies
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "statuses" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL UNIQUE,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "positions" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL UNIQUE,
        "salary" INTEGER NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "space_objects" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "airports" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "products" (
        "id" INTEGER NOT NULL UNIQUE,
        "product" VARCHAR NOT NULL,
        "price" INTEGER NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "reasons" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL UNIQUE,
        "cost" INTEGER NOT NULL,
        "product_id" INTEGER NOT NULL,
        "count" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("product_id") REFERENCES "products"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "spaceships" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "status_id" INTEGER NOT NULL,
        "human_places" INTEGER NOT NULL,
        "petrol_tank_size" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("status_id") REFERENCES "statuses"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "clients" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "surname" VARCHAR NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "tours" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "obj_id" INTEGER NOT NULL,
        "airport_id" INTEGER NOT NULL,
        "spaceship_id" INTEGER NOT NULL,
        "cost" INTEGER NOT NULL,
        "date" DATE NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("airport_id") REFERENCES "airports"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("spaceship_id") REFERENCES "spaceships"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("obj_id") REFERENCES "space_objects"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "employees" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "surname" VARCHAR NOT NULL,
        "tour_id" INTEGER,
        "position_id" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("tour_id") REFERENCES "tours"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("position_id") REFERENCES "positions"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "maintain_employees" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "surname" VARCHAR NOT NULL,
        "ticket_id" INTEGER DEFAULT NULL,
        "position_id" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("position_id") REFERENCES "positions"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "maintain_tickets" (
        "id" INTEGER NOT NULL UNIQUE,
        "spaceship_id" INTEGER NOT NULL,
        "reason_id" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("reason_id") REFERENCES "reasons"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("spaceship_id") REFERENCES "spaceships"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("id") REFERENCES "maintain_employees"("ticket_id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "tour_tickets" (
        "tour_id" INTEGER NOT NULL,
        "price" INTEGER NOT NULL,
        "client_id" INTEGER NOT NULL,
        FOREIGN KEY ("client_id") REFERENCES "clients"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("tour_id") REFERENCES "tours"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "reviews" (
        "text" VARCHAR NOT NULL,
        "client_id" INTEGER NOT NULL,
        "tour_id" INTEGER NOT NULL,
        "date" DATE NOT NULL,
        FOREIGN KEY ("client_id") REFERENCES "clients"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("tour_id") REFERENCES "tours"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "candidats" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "surname" VARCHAR NOT NULL,
        "position_id" INTEGER NOT NULL,
        "mark" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("position_id") REFERENCES "positions"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    # Commit the changes
    conn.commit()
    print("All tables created successfully in space_tourism.db!")

except sq.Error as e:
    print(f"An error occurred: {e}")

finally:
    # Close the connection
    conn.close()

All tables created successfully in space_tourism.db!


In [ ]:
conn.close()

In [ ]:
import sqlite3
import random
from datetime import datetime, timedelta

def generate_mock_data():
    conn = sqlite3.connect('space_tourism.db')
    cursor = conn.cursor()

    try:
        # Clear existing data (optional)
        cursor.execute("PRAGMA foreign_keys = OFF")
        tables = ["reviews", "tour_tickets", "maintain_employees", "maintain_tickets",
                "employees", "tours", "clients", "spaceships", "airports",
                "space_objects", "reasons", "products", "positions", "statuses",
                "candidats"]
        for table in tables:
            cursor.execute(f"DELETE FROM {table}")
        cursor.execute("PRAGMA foreign_keys = ON")

        # 1. Statuses
        statuses = [
            (1, "Ready"),
            (2, "Maintenance"),
            (3, "In Flight"),
            (4, "Retired")
        ]
        cursor.executemany("INSERT INTO statuses VALUES (?, ?)", statuses)

        # 2. Positions
        positions = [
            (1, "Pilot", 100000),
            (2, "Co-Pilot", 80000),
            (3, "Flight Attendant", 50000),
            (4, "Engineer", 90000),
            (5, "Mechanic", 60000),
            (6, "Tour Guide", 45000)
        ]
        cursor.executemany("INSERT INTO positions VALUES (?, ?, ?)", positions)

        # 3. Space Objects
        space_objects = [
            (1, "Moon"),
            (2, "Mars"),
            (3, "International Space Station"),
            (4, "Venus Orbit"),
            (5, "Jupiter Flyby")
        ]
        cursor.executemany("INSERT INTO space_objects VALUES (?, ?)", space_objects)

        # 4. Airports
        airports = [
            (1, "Starport Alpha"),
            (2, "Cosmodrome Beta"),
            (3, "Galactic Central"),
            (4, "Orbital One")
        ]
        cursor.executemany("INSERT INTO airports VALUES (?, ?)", airports)

        # 5. Products
        products = [
            (1, "Engine Parts", 5000),
            (2, "Life Support System", 12000),
            (3, "Navigation Computer", 8000),
            (4, "Hull Plating", 3000)
        ]
        cursor.executemany("INSERT INTO products VALUES (?, ?, ?)", products)

        # 6. Reasons
        reasons = [
            (1, "Engine Overhaul", 25000, 1, 1),
            (2, "Life Support Maintenance", 18000, 2, 1),
            (3, "Navigation System Upgrade", 15000, 3, 1),
            (4, "Hull Repair", 9000, 4, 3)
        ]
        cursor.executemany("INSERT INTO reasons VALUES (?, ?, ?, ?, ?)", reasons)

        # 7. Spaceships
        spaceships = [
            (1, "Star Voyager", 1, 50, 5000),
            (2, "Galactic Cruiser", 1, 75, 7500),
            (3, "Cosmic Explorer", 2, 30, 4000),
            (4, "Orbital Shuttle", 1, 20, 3000)
        ]
        cursor.executemany("INSERT INTO spaceships VALUES (?, ?, ?, ?, ?)", spaceships)

        # 8. Clients
        clients = []
        first_names = ["John", "Emma", "Michael", "Sophia", "William", "Olivia"]
        last_names = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia"]
        for i in range(1, 21):
            clients.append((
                i,
                random.choice(first_names),
                random.choice(last_names)
            ))
        cursor.executemany("INSERT INTO clients VALUES (?, ?, ?)", clients)

        # 9. Tours
        tours = []
        tour_names = [
            "Lunar Getaway",
            "Mars Expedition",
            "ISS Experience",
            "Venus Sightseeing",
            "Jupiter Adventure"
        ]
        start_date = datetime.now() + timedelta(days=10)
        for i in range(1, 11):
            tour_date = start_date + timedelta(days=random.randint(0, 90))
            tours.append((
                i,
                random.choice(tour_names),
                random.randint(1, 5),  # obj_id
                random.randint(1, 4),  # airport_id
                random.randint(1, 4),  # spaceship_id
                random.randint(5000, 20000),  # cost
                tour_date.strftime("%Y-%m-%d")  # date
            ))
        cursor.executemany("INSERT INTO tours VALUES (?, ?, ?, ?, ?, ?, ?)", tours)

        # 10. Employees
        employees = []
        emp_first_names = ["David", "Sarah", "Robert", "Jennifer", "James", "Lisa"]
        emp_last_names = ["Miller", "Davis", "Wilson", "Anderson", "Thomas", "Taylor"]
        for i in range(1, 16):
            employees.append((
                i,
                random.choice(emp_first_names),
                random.choice(emp_last_names),
                random.randint(1, 10) if random.random() > 0.3 else None,  # tour_id
                random.randint(1, 6)  # position_id
            ))
        cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?, ?)", employees)

        # 11. Maintain Employees
        maint_employees = []
        for i in range(1, 6):
            maint_employees.append((
                i,
                random.choice(emp_first_names),
                random.choice(emp_last_names),
                None,  # ticket_id will be set after maintain_tickets
                random.choice([4, 5])  # position_id (engineer or mechanic)
            ))
        cursor.executemany("INSERT INTO maintain_employees VALUES (?, ?, ?, ?, ?)", maint_employees)

        # 12. Maintain Tickets
        maintain_tickets = []
        for i in range(1, 6):
            maintain_tickets.append((
                i,
                random.randint(1, 4),  # spaceship_id
                random.randint(1, 4)  # reason_id
            ))
        cursor.executemany("INSERT INTO maintain_tickets VALUES (?, ?, ?)", maintain_tickets)

        # Update maintain_employees with ticket_ids
        for i in range(1, 6):
            cursor.execute("UPDATE maintain_employees SET ticket_id = ? WHERE id = ?", (i, i))

        # 13. Tour Tickets
        tour_tickets = []
        for i in range(1, 31):
            tour_tickets.append((
                random.randint(1, 10),  # tour_id
                random.randint(4500, 22000),  # price
                random.randint(1, 20)  # client_id
            ))
        cursor.executemany("INSERT INTO tour_tickets VALUES (?, ?, ?)", tour_tickets)

        # 14. Reviews
        reviews = []
        review_texts = [
            "Amazing experience!",
            "Once in a lifetime opportunity.",
            "The food could be better.",
            "Worth every penny!",
            "The crew was fantastic.",
            "Some technical issues during flight."
        ]
        for i in range(1, 21):
            review_date = (datetime.now() - timedelta(days=random.randint(1, 30))).strftime("%Y-%m-%d")
            reviews.append((
                random.choice(review_texts),
                random.randint(1, 20),  # client_id
                random.randint(1, 10),  # tour_id
                review_date
            ))
        cursor.executemany("INSERT INTO reviews VALUES (?, ?, ?, ?)", reviews)

        # 15. Candidates
        candidates = []
        for i in range(1, 11):
            candidates.append((
                i,
                random.choice(emp_first_names),
                random.choice(emp_last_names),
                random.randint(1, 6),  # position_id
                random.randint(3, 10)  # mark
            ))
        cursor.executemany("INSERT INTO candidats VALUES (?, ?, ?, ?, ?)", candidates)

        conn.commit()
        print("Successfully generated and inserted mock data!")

    except sqlite3.Error as e:
        print(f"Error generating mock data: {e}")
    finally:
        conn.close()

if __name__ == "__main__":
    generate_mock_data()

Successfully generated and inserted mock data!


In [ ]:
conn.close()

In [ ]:
conn = sq.connect('space_tourism.db')
cursor = conn.cursor()

In [ ]:
list(cursor.execute('''
SELECT
    t.name AS tour_name,
    GROUP_CONCAT(DISTINCT t.date) AS tour_date,
    AVG(tt.price) AS avg_ticket_price,
    COUNT(tt.client_id) AS client_count
FROM tours t
JOIN tour_tickets tt ON t.id = tt.tour_id
WHERE t.date > '2024-01-01'
GROUP BY t.name
HAVING COUNT(tt.client_id) > 2
ORDER BY avg_ticket_price DESC;

'''))

[('Mars Expedition', '2025-07-15,2025-07-05', 17365.5, 4),
 ('Venus Sightseeing', '2025-05-23,2025-06-25,2025-06-07', 12655.5, 10),
 ('Lunar Getaway', '2025-07-29,2025-07-18', 12336.57142857143, 7),
 ('Jupiter Adventure', '2025-07-08,2025-08-13', 11216.714285714286, 7)]

In [ ]:
list(cursor.execute('''
SELECT
    c.name || ' ' || c.surname AS client_name,
    r.text AS review_text,
    r.date AS review_date,
    t.name AS tour_name,
    AVG(tt.price) AS avg_tour_price
FROM clients c
JOIN reviews r ON c.id = r.client_id
JOIN tours t ON r.tour_id = t.id
JOIN tour_tickets tt ON t.id = tt.tour_id
GROUP BY c.id
HAVING avg_tour_price > (SELECT AVG(price) FROM tour_tickets)
ORDER BY avg_tour_price DESC
LIMIT 10;


'''))

[('William Smith',
  'Once in a lifetime opportunity.',
  '2025-04-14',
  'Venus Sightseeing',
  16448.833333333332),
 ('Olivia Williams',
  'Amazing experience!',
  '2025-04-26',
  'Mars Expedition',
  15206.0),
 ('Sophia Brown',
  'Some technical issues during flight.',
  '2025-04-18',
  'Mars Expedition',
  15206.0),
 ('Emma Smith',
  'The food could be better.',
  '2025-04-09',
  'Venus Sightseeing',
  14910.75),
 ('John Johnson',
  'Some technical issues during flight.',
  '2025-05-07',
  'Venus Sightseeing',
  14910.75),
 ('John Brown',
  'Amazing experience!',
  '2025-05-06',
  'Venus Sightseeing',
  13171.888888888889)]

In [ ]:
list(cursor.execute('''
SELECT
    c.name || ' ' || c.surname AS candidate_name,
    p.name AS position_name,
    c.mark AS candidate_mark,
    AVG(c2.mark) AS avg_position_mark,
    COUNT(e.id) AS current_employees_in_position
FROM candidats c
JOIN positions p ON c.position_id = p.id
LEFT JOIN employees e ON e.position_id = p.id
JOIN candidats c2 ON c2.position_id = p.id
GROUP BY c.id
HAVING c.mark > (SELECT AVG(mark) FROM candidats WHERE position_id = p.id)
ORDER BY c.mark DESC;


'''))

[('Jennifer Anderson', 'Mechanic', 10, 6.5, 2),
 ('Jennifer Miller', 'Pilot', 10, 7.0, 6),
 ('Lisa Davis', 'Tour Guide', 10, 7.5, 6),
 ('James Thomas', 'Co-Pilot', 7, 5.0, 12)]

In [ ]:
list(cursor.execute('''
SELECT
    s.name AS spaceship_name,
    SUM(p.price * r.count) AS total_maintenance_cost
FROM spaceships s
JOIN maintain_tickets mt ON s.id = mt.spaceship_id
JOIN reasons r ON mt.reason_id = r.id
JOIN products p ON r.product_id = p.id
GROUP BY s.id
ORDER BY total_maintenance_cost DESC;


'''))

[('Orbital Shuttle', 23000), ('Galactic Cruiser', 17000)]

In [ ]:
list(cursor.execute('''
SELECT
    a.name AS airport_name,
    COUNT(t.id) AS tour_count,
    GROUP_CONCAT(DISTINCT so.name) AS space_objects,
    SUM(s.human_places) AS total_passenger_capacity
FROM airports a
JOIN tours t ON a.id = t.airport_id
JOIN space_objects so ON t.obj_id = so.id
JOIN spaceships s ON t.spaceship_id = s.id
GROUP BY a.id
ORDER BY tour_count DESC;

'''))

[('Orbital One',
  6,
  'Moon,Jupiter Flyby,International Space Station,Mars',
  235),
 ('Galactic Central', 2, 'Moon,Jupiter Flyby', 125),
 ('Cosmodrome Beta', 1, 'Venus Orbit', 50),
 ('Starport Alpha', 1, 'International Space Station', 75)]

In [ ]:
list(cursor.execute('''
SELECT
    s.name AS spaceship_name,
    st.name AS status,
    s.human_places AS seats,
    GROUP_CONCAT(DISTINCT t.name) AS tours
FROM spaceships s
JOIN statuses st ON st.id = s.status_id
JOIN tours t ON s.id = t.spaceship_id
GROUP BY s.id;

'''))

[('Star Voyager',
  'Ready',
  50,
  'Venus Sightseeing,Mars Expedition,Jupiter Adventure'),
 ('Galactic Cruiser',
  'Ready',
  75,
  'Mars Expedition,Lunar Getaway,Venus Sightseeing'),
 ('Orbital Shuttle',
  'Ready',
  20,
  'Jupiter Adventure,ISS Experience,Lunar Getaway')]

In [ ]:
list(cursor.execute('''
SELECT
    e.name || ' ' || e.surname AS employee_name,
    p.name AS position,
    p.salary AS salary,
    t.name AS assigned_tour,
    t.date AS tour_date
FROM employees e
JOIN positions p ON e.position_id = p.id
LEFT JOIN tours t ON e.tour_id = t.id
ORDER BY p.salary DESC;
'''))

[('James Thomas', 'Pilot', 100000, 'Venus Sightseeing', '2025-06-25'),
 ('Jennifer Wilson', 'Pilot', 100000, None, None),
 ('Jennifer Anderson', 'Engineer', 90000, 'Venus Sightseeing', '2025-06-25'),
 ('David Anderson', 'Engineer', 90000, None, None),
 ('James Miller', 'Co-Pilot', 80000, 'Jupiter Adventure', '2025-07-08'),
 ('David Taylor', 'Co-Pilot', 80000, None, None),
 ('Jennifer Thomas', 'Co-Pilot', 80000, None, None),
 ('Sarah Wilson', 'Co-Pilot', 80000, 'Venus Sightseeing', '2025-06-07'),
 ('Sarah Anderson', 'Mechanic', 60000, 'ISS Experience', '2025-08-04'),
 ('Jennifer Wilson',
  'Flight Attendant',
  50000,
  'Venus Sightseeing',
  '2025-06-25'),
 ('David Thomas', 'Flight Attendant', 50000, 'ISS Experience', '2025-08-04'),
 ('Lisa Anderson', 'Flight Attendant', 50000, None, None),
 ('Sarah Anderson', 'Tour Guide', 45000, 'Jupiter Adventure', '2025-08-13'),
 ('Jennifer Anderson', 'Tour Guide', 45000, None, None),
 ('David Davis', 'Tour Guide', 45000, 'Venus Sightseeing', '2025

In [ ]:
list(cursor.execute('''
SELECT
    so.name AS destination,
    COUNT(t.id) AS tour_count,
    COUNT(DISTINCT tt.client_id) AS unique_clients
FROM space_objects so
JOIN tours t ON so.id = t.obj_id
JOIN tour_tickets tt ON t.id = tt.tour_id
GROUP BY so.id
ORDER BY tour_count DESC;

'''))

[('International Space Station', 10, 8),
 ('Jupiter Flyby', 9, 8),
 ('Moon', 6, 5),
 ('Venus Orbit', 3, 3),
 ('Mars', 2, 1)]

In [ ]:
list(cursor.execute('''
    SELECT
    t.name AS tour,
    so.name AS destination,
    t.cost AS base_price,
    AVG(tt.price) AS actual_avg_price,
    AVG(tt.price) - t.cost AS price_difference,
    CASE
        WHEN AVG(tt.price) > t.cost THEN 'Increased'
        ELSE 'Decreased'
    END AS pricing_strategy
FROM tours t
JOIN space_objects so ON t.obj_id = so.id
JOIN tour_tickets tt ON t.id = tt.tour_id
GROUP BY t.id;
'''))

[('Venus Sightseeing', 'Moon', 14975, 14910.75, -64.25, 'Decreased'),
 ('Mars Expedition', 'Jupiter Flyby', 18685, 15206.0, -3479.0, 'Decreased'),
 ('Jupiter Adventure',
  'International Space Station',
  13521,
  11780.8,
  -1740.2000000000007,
  'Decreased'),
 ('ISS Experience', 'Mars', 8716, 10390.5, 1674.5, 'Increased'),
 ('Mars Expedition', 'Moon', 6767, 19525.0, 12758.0, 'Increased'),
 ('Lunar Getaway',
  'Jupiter Flyby',
  7629,
  12234.333333333334,
  4605.333333333334,
  'Increased'),
 ('Jupiter Adventure',
  'International Space Station',
  19690,
  9806.5,
  -9883.5,
  'Decreased'),
 ('Venus Sightseeing',
  'Venus Orbit',
  17689,
  11062.666666666666,
  -6626.333333333334,
  'Decreased'),
 ('Lunar Getaway', 'Jupiter Flyby', 15309, 12413.25, -2895.75, 'Decreased'),
 ('Venus Sightseeing',
  'International Space Station',
  10916,
  11241.333333333334,
  325.33333333333394,
  'Increased')]

In [ ]:
list(cursor.execute('''
SELECT
    p.name AS position,
    COUNT(e.id) AS employee_count,
    p.salary AS position_salary,
    COUNT(e.id) * p.salary AS total_salary_cost,
    ROUND(COUNT(e.id) * 100.0 / (SELECT COUNT(*) FROM employees), 1) AS percentage
FROM positions p
LEFT JOIN employees e ON p.id = e.position_id
GROUP BY p.id
ORDER BY total_salary_cost DESC;
'''))

[('Co-Pilot', 4, 80000, 320000, 26.7),
 ('Pilot', 2, 100000, 200000, 13.3),
 ('Engineer', 2, 90000, 180000, 13.3),
 ('Flight Attendant', 3, 50000, 150000, 20.0),
 ('Tour Guide', 3, 45000, 135000, 20.0),
 ('Mechanic', 1, 60000, 60000, 6.7)]